## Machine Learning with MLlib
## *Introduction and Feature Extraction*

### University of California, Santa Barbara  
### PSTAT 135/235  
### Last Updated: Dec 12, 2018

---  

### Sources 

1. Learning Spark
2. Spark Documentation  
	https://spark.apache.org/docs/latest/mllib-data-types.html  
	http://spark.apache.org/docs/1.2.0/mllib-feature-extraction.html

### OBJECTIVES
1. Introduction to the machine learning library
2. Introduction to MLlib data types
3. Discuss Feature Extraction tools in MLLib


### CONCEPTS AND FUNCTIONS
- pipeline  
- supervised and unsupervised learning  
- learning tasks: classification, regression, clustering, dimensionality reduction  
- training set, testing set  
- feature extraction  

- MLlib data types:  
  - LabeledPoint  
  - sparse vector, dense vector  
  - sparse matrix, dense matrix  
  - Rating  

- Feature Extraction  
- TF-IDF  
- Word2Vec  
- Cosine Similarity  


---  

**MLlib**

Contains Spark’s ML library  
Works on RDDs  
Contains only algorithms that can be parallelized, since those run well on clusters  
Includes a pipeline API useful for building ML pipelines, similar to scikit-learn  


### Build LogReg Classifier to Predict Spam vs Not

In [12]:
data_path = '/home/jovyan/UCSB_BigDataAnalytics/data/mllib/'

In [17]:
import os
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("mllib_classifier") \
        .getOrCreate()

In [15]:
spark

In [16]:
sc = spark.sparkContext

In [18]:
spam = sc.textFile(os.path.join(data_path, "spam.txt"))
ham = sc.textFile(os.path.join(data_path, "ham.txt"))

In [ ]:
spam.collect()

In [ ]:
ham.collect()

In [22]:
tf = HashingTF(numFeatures = 10000)

In [23]:
spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
normalFeatures = ham.map(lambda email: tf.transform(email.split(" ")))

In [24]:
# Build LabeledPoint datasets (1=spam, 0=ham)
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = normalFeatures.map(lambda features: LabeledPoint(0, features))

In [25]:
pos = positiveExamples.collect()

In [ ]:
pos[0]

In [28]:
neg = negativeExamples.collect()

In [ ]:
neg[0]

In [ ]:
# Build training set
trainData = positiveExamples.union(negativeExamples)
trainData.cache()

In [31]:
# Train LogReg model
model = LogisticRegressionWithSGD.train(trainData)

In [ ]:
# push "not spam" example through classifier
posTest = tf.transform("I love learning Spark programming".split(" "))
posTest

In [38]:
# Prediction
print("Prediction for positive example: {}".format(model.predict(posTest)))

Prediction for positive example: 0


**LabeledPoint**  
Stores feature vector together with label  
**Rating**  
Rating of product by a user. Used in recommendation, for instance.  
**Vector**  
Handles dense and sparse. For sparse, only nonzero values and their indices are stored.  
Sparse saves on memory and runtime.  
**Matrix**  
A local matrix has integer-typed row and column indices and double-typed values, stored on a single   machine.  
MLlib supports dense matrices, whose entry values are stored in a single double array in column-major order, and sparse matrices, whose non-zero entry values are stored in the Compressed Sparse Column (CSC) format in column-major order.  
**Distributed matrix**  
A distributed matrix has long-typed row and column indices and double-typed values  
**Row matrix**  
A RowMatrix is a row-oriented distributed matrix without meaningful row indices  
**CoordinateMatrix**  
CoordinateMatrix is a distributed matrix backed by an RDD of its entries  
A CoordinateMatrix should be used only when both dimensions of the matrix are huge and the matrix is very sparse.

https://en.wikipedia.org/wiki/Row-_and_column-major_order

In [41]:
# Build sparse vector
from pyspark.mllib.linalg import Vectors

# create sparse vector [1.0 0.0 2.0 0.0]
sv1 = Vectors.sparse(4, {0: 1.0, 2: 2.0})

In [ ]:
sv1

### Feature Extraction

*mllib.feature*  
contains classes for common feature transformations:  
-  Term Frequency-Inverse Document Frequency (TF-IDF)  
Produces feature vectors from text documents

There are two algorithms that compute TF-IDF:  

**1. HashingTF**  
	Computes term frequency vector from document  
	Can process one document or an RDD of documents  
	Each document needs to be an interable sequence (a list in Python)  

To reduce the chance of collision, we can increase the target feature dimension, i.e., the  
	 number of buckets of the hash table. The default feature dimension is 1,048,576  

**2. IDF**  
	Computes inverse document frequency  
	Terms that appear in high fraction of the docs are not as valuable  
	IDF will downweight such terms  

Good example of Feature Extraction here:  
http://spark.apache.org/docs/1.2.0/mllib-feature-extraction.html

**TF-IDF Example**

**Word2Vec**  
Computes distributed vector representation of words.  
Similar words are close in the vector space  
Useful in many NLP applications:  
named entity recognition, disambiguation, parsing, tagging and machine translation.  

### Fit Word2VecModel to some text data

In [ ]:
from pyspark.mllib.feature import Word2Vec

inp = sc.textFile("C:/spark/spark-2.1.1-bin-hadoop2.7/data/text8_part1.txt").map(lambda row: row.split(" "))
word2vec = Word2Vec()
model = word2vec.fit(inp)

synonyms = model.findSynonyms('china', 40)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))
Top Records:
malaysia: 0.9055396899188917
eastern: 0.8834685956632131
africa: 0.8537198739068056
zambia: 0.8535407384161012
myanmar: 0.8475548784366893
predominantly: 0.8461926971224027
mongolia: 0.8371518611342739
countries: 0.8342705781501009
southeast: 0.8316274754770454
central: 0.8313670331856243

**StandardScaler**   

Standardization can improve the convergence rate during the optimization process, and also  
prevents against features with very large variances exerting an overly large influence during model   training.  

For each feature,  
1. Scales to unit variance  
2. Centers to mean zero  
Useful or even essential for some models  
K-means works in Euclidean space, so all features should be on same scale  
Tree models do not need this

Use this in a *Pipeline* so the statistics can be applied to datasets for scoring later

### Standard Scaler  
Load dataset in libsvm format, standardize the features so that the new features have unit variance and/or zero mean

In [43]:
from pyspark.mllib.util import MLUtils
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

In [47]:
data = MLUtils.loadLibSVMFile(sc, os.path.join(data_path, "sample_libsvm_data.txt"))

In [ ]:
data.take(1)

In [ ]:
type(data)

In [51]:
# extract labels and features; stored as RDDs
label = data.map(lambda x: x.label)
features = data.map(lambda x: x.features)

In [53]:
scaler1 = StandardScaler().fit(features)

In [54]:
# data1 will be unit variance.
data1 = label.zip(scaler1.transform(features))

In [ ]:
data1.take(2)